# Intento 3

| Score | Timestamp |
| --- | --- |
| **0.8412** | 2021-12-31 02:09:09 UTC |

- Cambio de `n_estimators` de LightGBM a 50.

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.multioutput import MultiOutputClassifier
import lightgbm as lgb

# código común
import common.preprocessing as pre
from common.vars import SEED

# display all columns
pd.options.display.max_columns = None

In [2]:
train_features = pd.read_csv('../data/flu_training_set_features.csv')
train_labels = pd.read_csv('../data/flu_training_set_labels.csv')
test_features = pd.read_csv('../data/flu_test_set_features.csv')

In [3]:
# no usamos el respondent_id
train_features.drop(labels=['respondent_id'], axis=1, inplace=True)
train_labels.drop(labels=['respondent_id'], axis=1, inplace=True)
test_features.drop(labels=['respondent_id'], axis=1, inplace=True)

In [4]:
def preprocess(df_train, df_test, pre_onehot=True):
  """PREPROCESAMIENTO
  """
  _df_train = df_train.copy()
  _df_test = df_test.copy()
  # antes de realizar one-hot encoding, eliminamos los valores perdidos de atributos categóricos
  # lo haremos imputando a la clase mayoritaria (mode)
  if pre_onehot:
    attrs_imp = ['race', 'employment_status', 'census_msa', 'hhs_geo_region', 'employment_industry', 'employment_occupation']
    _df_train = pre.impute(_df_train, strategy='mode', attrs=attrs_imp)
    _df_test = pre.impute(_df_test, strategy='mode', attrs=attrs_imp)
  
  def trans(df):
    df = pre.trans_onehot(df, 'race')
    df = pre.trans_onehot(df, 'employment_status')
    df = pre.trans_onehot(df, 'census_msa')
    df = pre.trans_discretize(df, 'age_group', {
      '18 - 34 Years':  1,
      '35 - 44 Years':  2,
      '45 - 54 Years':  3,
      '55 - 64 Years':  4,
      '65+ Years':      5
    })
    df = pre.trans_discretize(df, 'education', {
      '< 12 Years':       1,
      '12 Years':         2,
      'Some College':     3,
      'College Graduate': 4
    })
    df = pre.trans_discretize(df, 'sex', {
      'Male':   0,
      'Female': 1
    })
    df = pre.trans_discretize(df, 'income_poverty', {
      'Below Poverty':              1,
      '<= $75,000, Above Poverty':  2,
      '> $75,000':                  3
    })
    df = pre.trans_discretize(df, 'marital_status', {
      'Not Married':  0,
      'Married':      1
    })
    df = pre.trans_discretize(df, 'rent_or_own', {
      'Rent': 0,
      'Own':  1
    })
    return df
  _df_test = trans(_df_test)
  _df_train = trans(_df_train)

  # discretizamos estas columnas asignando un valor discreto cualquiera, secuencialmente
  # (hacemos la labor de LabelEncoder para los atributos)
  for attr in ['hhs_geo_region', 'employment_industry', 'employment_occupation']:
    _df_train, discrete_dict = pre.trans_discretize(_df_train, attr)
    # añadimos las mismas claves que df_train
    _df_test = pre.trans_discretize(_df_test, attr, discrete_dict)
  
  # VALORES PERDIDOS
  # del resto de atributos, imputamos la mediana (median)
  _df_train = pre.impute(_df_train, strategy='median')
  _df_test = pre.impute(_df_test, strategy='median')
  
  return _df_train, _df_test

In [5]:
train_features, test_features = preprocess(train_features, test_features)

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

def cross_validation(model, X, y, cv):
  measures = []
  time_total_0 = time.time()

  for train, test in cv.split(X, y):
    X_train = X.loc[train, :].values
    X_test = X.loc[test, :].values
    y_train = y.loc[train, :].values
    y_test = y.loc[test, :].values
    time_iter_0 = time.time()
    _model = model.fit(X_train, y_train)
    time_iter = time.time() - time_iter_0
    preds = _model.predict_proba(X_test)
    y_preds = pd.DataFrame(
      {
        'h1n1_vaccine':     preds[0][:,1],
        'seasonal_vaccine': preds[1][:,1],
      },
      index = y.loc[test,:].index
    )
    measure = roc_auc_score(y_test, y_preds)
    print(f"[CV] AUC score: {measure:.4f}. Time: {time_iter:6.2f}")
    measures.append(measure)
  
  time_total = time.time() - time_total_0
  print(f"[CV] Average AUC: {np.mean(measures):.4f}. Total time: {time_total:6.2f}")
  return model, y_preds

def compile_submission(preds, route):
  df_submission = pd.read_csv('../data/submission_format.csv')
  y_test_preds = pd.DataFrame(
    {
      'h1n1_vaccine': preds[0][:,1],
      'seasonal_vaccine': preds[1][:,1]
    },
    index = df_submission.index
  )
  df_submission['h1n1_vaccine'] = y_test_preds.h1n1_vaccine
  df_submission['seasonal_vaccine'] = y_test_preds.seasonal_vaccine
  df_submission.to_csv(route, index=False)

In [7]:
# LightGBM
lgbm = lgb.LGBMClassifier(objective='binary', n_estimators=50)
multi_lgbm = MultiOutputClassifier(lgbm)
lgbm_cvd, y_preds_lgbm = cross_validation(multi_lgbm, train_features, train_labels, kf)

[CV] AUC score: 0.8542. Time:   0.37
[CV] AUC score: 0.8521. Time:   0.40
[CV] AUC score: 0.8443. Time:   0.34
[CV] AUC score: 0.8520. Time:   0.39
[CV] AUC score: 0.8475. Time:   0.37
[CV] Average AUC: 0.8500. Total time:   2.14


In [8]:
# aprendemos con todos los ejemplos
lgbm_cvd = lgbm_cvd.fit(train_features, train_labels)
# aplicamos probabilidad
lgbm_preds = lgbm_cvd.predict_proba(test_features)

In [9]:
compile_submission(lgbm_preds, '../submissions/03.csv')